In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [2]:
df = np.array(df)
m, n =df.shape
np.random.shuffle(df)

df_dev = df[0:1000].T
y_dev= df_dev[0]
X_dev = df_dev[1:n]
X_dev = X_dev / 255

df_train = df[1000:m].T
y_train= df_train[0]
x_train = df_train[1:n]
x_train = x_train / 255
_,m_train = x_train.shape

In [3]:
x_train.shape

(784, 41000)

In [4]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) -0.5
    b2 = np.random.rand(10, 1) -0.5
    return W1, b1, W2, b2
    
def ReLu(Z):
    A = np.maximum(Z, 0)
    return A
def softmax(Z):
    Z = Z - np.max(Z, axis=0, keepdims=True)  # Subtract maximum value for stability
    exp_Z = np.exp(Z)
    A = exp_Z / np.sum(exp_Z, axis=0, keepdims=True)
    return A
   
    
def forward_prop(W1, b1, W2, b2, X):
    Z1=W1.dot(X)+b1
    A1=ReLu(Z1)
    Z2=W2.dot(A1)+b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2
    
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() +1))
    one_hot_Y[np.arange(Y.size), Y.astype(int)] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLu(Z):
    return Z > 0
    
def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y=one_hot(Y)
    dZ2=A2-one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2= 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1= W2.T.dot(dZ2)*deriv_ReLu(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1= 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(x_train, y_train, 0.3, 2000)

Iteration:  0
[1 1 1 ... 1 1 1] [1 1 4 ... 6 1 6]
0.11746341463414633
Iteration:  10
[5 1 6 ... 6 5 3] [1 1 4 ... 6 1 6]
0.40370731707317076
Iteration:  20
[1 1 6 ... 6 1 6] [1 1 4 ... 6 1 6]
0.5443170731707317
Iteration:  30
[1 1 4 ... 6 1 6] [1 1 4 ... 6 1 6]
0.613609756097561
Iteration:  40
[1 1 4 ... 6 1 6] [1 1 4 ... 6 1 6]
0.6611463414634147
Iteration:  50
[1 1 4 ... 6 1 6] [1 1 4 ... 6 1 6]
0.699560975609756
Iteration:  60
[1 1 4 ... 6 1 6] [1 1 4 ... 6 1 6]
0.7298780487804878
Iteration:  70
[1 1 4 ... 6 1 6] [1 1 4 ... 6 1 6]
0.7543414634146342
Iteration:  80
[1 1 4 ... 6 1 6] [1 1 4 ... 6 1 6]
0.7412926829268293
Iteration:  90
[1 1 9 ... 6 1 6] [1 1 4 ... 6 1 6]
0.7545121951219512
Iteration:  100
[1 1 9 ... 6 1 6] [1 1 4 ... 6 1 6]
0.7823170731707317
Iteration:  110
[1 1 9 ... 6 1 6] [1 1 4 ... 6 1 6]
0.7945365853658537
Iteration:  120
[1 1 9 ... 6 1 6] [1 1 4 ... 6 1 6]
0.8039024390243903
Iteration:  130
[1 1 9 ... 6 1 6] [1 1 4 ... 6 1 6]
0.8140731707317073
Iteration:  140
[

In [7]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2 , X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = x_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = y_train[index]
    print("Prédiction: ", prediction)
    print("label: ", label)



In [8]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, y_dev)

[7 9 7 7 4 7 0 2 9 5 2 7 7 3 5 2 7 5 3 1 2 4 2 6 5 9 5 0 6 0 8 4 7 7 1 2 0
 0 1 5 6 7 8 4 6 4 6 5 0 7 4 8 7 1 9 0 6 7 0 4 9 7 1 4 8 9 0 8 2 8 7 6 0 7
 9 3 0 2 4 6 1 0 1 4 0 4 8 1 3 8 6 7 1 7 7 2 2 6 8 6 7 8 8 8 6 8 4 1 0 7 9
 1 8 4 2 4 0 0 4 8 4 4 7 7 8 8 4 1 9 7 1 8 1 2 4 3 6 5 7 6 1 4 4 5 1 8 9 1
 9 4 3 2 9 9 8 5 5 1 4 8 8 8 0 5 6 8 5 8 6 2 7 8 1 3 7 9 2 7 7 5 5 7 7 5 1
 4 1 1 2 5 1 7 6 4 5 0 1 6 9 0 6 4 1 1 6 8 6 7 9 5 8 5 4 8 4 6 9 1 0 1 0 4
 0 5 0 3 4 5 6 4 0 3 7 3 9 5 7 2 6 8 5 7 2 6 2 5 9 7 1 5 2 8 3 1 3 3 6 8 5
 9 5 4 2 3 5 8 4 3 8 6 7 8 6 7 8 7 8 1 4 3 3 8 7 4 6 0 3 3 7 6 0 5 5 7 0 1
 1 3 6 1 8 4 4 9 2 4 0 6 3 2 2 4 7 4 5 6 8 2 5 3 8 1 6 0 7 2 5 3 5 1 7 4 5
 1 5 5 3 3 8 2 4 9 0 9 6 5 6 1 6 3 5 7 3 1 4 6 5 2 5 0 4 5 6 0 3 2 2 2 5 7
 5 1 8 1 5 1 2 5 2 8 5 0 0 5 6 9 1 1 1 4 4 1 6 1 8 9 8 9 4 4 4 4 2 1 4 4 7
 7 9 7 5 4 7 3 6 3 5 8 5 5 6 1 0 0 1 7 3 9 2 6 4 9 0 1 1 3 4 3 2 2 7 1 1 7
 9 7 7 5 6 0 6 7 0 4 1 3 1 0 7 7 5 6 2 2 1 8 1 3 8 8 1 2 0 2 9 8 5 0 0 2 8
 3 4 7 1 0 5 1 7 2 8 6 2 

0.913